In [52]:
from patient_info import *
from toolset.io_handler.logger import check_path_with_log

import os
os.getcwd()

'/Users/iris/Desktop/LCPF/toolset/case_handler'

In [37]:
# read table: one patient per row
expression_root_dir = tcga_path + "/gene_expression_and_mutation/"
assmbled_info = get_assembled_table()
assmbled_info.head()

,Unnamed: 0.1,Unnamed: 0,Cart,Case UUID,Case ID,Primary Site,Gender,Files,Seq,Exp,...,Program,Disease Type,Age at diagnosis,Days to death,Vital Status,Primary Diagnosis,Ethnicity,Race,RNA_Seq,WXS
0,0,0,Case Actions,a5f2576a-d628-47de-a5df-76a4f3105dcc,TCGA-L9-A743,Bronchus and lung,Male,69,6,4,...,TCGA,Adenomas and Adenocarcinomas,56 years 326 days,--,Alive,"Adenocarcinoma, NOS",not hispanic or latino,black or african american,51847dea-489f-4df9-84f8-26a6f2579620/35a9d225-...,4b20d50c-f569-4703-93ec-584c267e3fdf/0ef89e1c-...
1,1,1,Case Actions,c4c1469d-752f-42b6-8686-714ad4a8ac97,TCGA-75-6206,Bronchus and lung,Male,70,8,4,...,TCGA,Adenomas and Adenocarcinomas,--,--,Alive,"Adenocarcinoma, NOS",not reported,not reported,9123a775-d686-49ab-90e7-0dc572c2fcaf/7643fc1a-...,1ce62e4b-ba0b-4410-a508-569d44de1f4b/7bba1178-...
2,2,2,Case Actions,8d0736fe-261c-445c-bfd2-a3ea3ceaf367,TCGA-55-6971,Bronchus and lung,Female,79,9,6,...,TCGA,Adenomas and Adenocarcinomas,59 years 185 days,--,Alive,"Adenocarcinoma, NOS",not reported,white,c36f84e5-8c59-46fe-af32-7f70684383b2/07a66ae7-...,99eeb9be-e2d0-48e3-9940-daf1aee705e4/8383e3da-...
3,3,3,Case Actions,33c71ead-4c2a-476d-85ef-885dbbfc2cbd,TCGA-44-6774,Bronchus and lung,Female,75,9,4,...,TCGA,Adenomas and Adenocarcinomas,56 years 8 days,--,Alive,"Adenocarcinoma, NOS",not hispanic or latino,white,3cd897bd-893e-4497-b9a5-56141816cff4/525e9185-...,80f7616d-b100-45a1-92c5-e804690ccbd6/3fa65836-...
4,4,4,Case Actions,f77b6930-a1ce-446e-a15d-c018cbbecfee,TCGA-49-4510,Bronchus and lung,Female,79,10,4,...,TCGA,Adenomas and Adenocarcinomas,51 years 135 days,2 years 166 days,Dead,"Adenocarcinoma, NOS",not hispanic or latino,black or african american,951f2356-17f4-4f4f-a8ec-855c7e93dce1/7eab2708-...,0da50c40-ffa7-4bfc-8e87-02d456940500/285301af-...


In [48]:
def encode_expression(exp_dir):
    # read expression data
    expression_table = pd.read_csv(exp_dir, sep="\t")
    print(exp_dir)


In [55]:
def process_row(patient):
    # get basic info
    #print(patient)

    # get expression file path
    exp_dir = expression_root_dir + patient["RNA_Seq"]
    exist_flag, message = check_path_with_log(exp_dir)
    print(message) # log later

    # get content
    if exist_flag:
        expression_df = pd.read_csv(exp_dir, skiprows=1, sep="\t")


    return expression_df




In [59]:
# for every patient
for index, patient in assmbled_info[0:1].iterrows():
    encoded = process_row(patient)

encoded.head(10)


VALID FILE: /Users/iris/Desktop/LCPF/tcga_db//gene_expression_and_mutation/51847dea-489f-4df9-84f8-26a6f2579620/35a9d225-0988-4e31-9a39-d54f3a63bea8.rna_seq.augmented_star_gene_counts.tsv


,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
0,N_unmapped,NaN,NaN,3485913,3485913,3485913,NaN,NaN,NaN
1,N_multimapping,NaN,NaN,6112198,6112198,6112198,NaN,NaN,NaN
2,N_noFeature,NaN,NaN,3489107,28212146,28316385,NaN,NaN,NaN
3,N_ambiguous,NaN,NaN,5256165,1403606,1400147,NaN,NaN,NaN
4,ENSG00000000003.15,TSPAN6,protein_coding,2559,1279,1280,35.4834,13.1692,12.9690
5,ENSG00000000005.6,TNMD,protein_coding,1,1,0,0.0426,0.0158,0.0156
6,ENSG00000000419.13,DPM1,protein_coding,1415,752,663,73.7356,27.3661,26.9499
7,ENSG00000000457.14,SCYL3,protein_coding,561,444,451,5.1264,1.9026,1.8737
8,ENSG00000000460.17,C1orf112,protein_coding,336,328,369,3.5399,1.3138,1.2938
9,ENSG00000000938.13,FGR,protein_coding,1778,894,884,33.0663,12.2722,12.0855
